In [74]:
cd "/Users/TaihuaLi/Desktop/Projects/Netflix Prize/"

/Users/TaihuaLi/Desktop/Projects/Netflix Prize


In [75]:
import numpy as np
import pandas as pd

In [89]:
ratings = pd.read_csv("Rating Zeros.csv", index_col=0) # row: users  col: movies
items = pd.read_csv("items.csv", index_col=0) # movies information
genre = pd.read_csv("genre.csv") # movie genre information

In [90]:
ratings.head(3)

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
items.head(3)

,Name,Unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantacy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
﻿Movie.ID,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [92]:
genre.head(3)

,﻿Genre,ID
0,unknown,0
1,Action,1
2,Adventure,2


### K-means Clustering: Users

In [80]:
from sklearn.cluster import KMeans

In [208]:
kmeans = KMeans(n_clusters=5)

In [209]:
kmeans.fit(ratings,)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [210]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [211]:
pd.set_option('display.max_rows', ratings.describe().shape[1])
pd.DataFrame(centroids)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.969325,0.018405,0.460123,0.134969,0.110429,0.110429,2.398773,0.147239,1.650307,0.392638,...,-5.637851e-18,-1.040834e-17,1.840491e-02,1.226994e-02,-5.637851e-18,-2.602085e-18,-5.637851e-18,-5.204170e-18,-5.637851e-18,-5.637851e-18
1,0.537859,0.023499,0.013055,0.052219,0.104439,0.067885,0.469974,0.206266,0.637076,0.164491,...,2.341877e-17,-1.214306e-17,2.341877e-17,-6.071532e-18,2.341877e-17,2.610966e-03,7.832898e-03,5.221932e-03,2.341877e-17,2.341877e-17
2,3.642857,2.214286,1.091837,2.693878,1.387755,0.051020,3.153061,2.336735,2.071429,0.612245,...,-5.637851e-18,-8.673617e-18,-5.637851e-18,-4.336809e-18,-5.637851e-18,-2.168404e-18,-5.637851e-18,-4.336809e-18,3.061224e-02,3.061224e-02
3,2.461538,0.723077,0.538462,2.538462,0.415385,0.507692,2.753846,2.815385,2.707692,1.569231,...,-3.035766e-18,6.153846e-02,-3.035766e-18,-3.469447e-18,-3.035766e-18,-1.734723e-18,-3.035766e-18,-3.469447e-18,-3.035766e-18,-3.035766e-18
4,2.333333,0.615385,0.217949,1.158120,0.269231,0.047009,1.837607,1.538462,1.166667,0.222222,...,1.282051e-02,-1.127570e-17,9.974660e-18,-5.637851e-18,1.282051e-02,-2.818926e-18,9.974660e-18,-5.637851e-18,9.974660e-18,9.974660e-18


In [212]:
movie_names = items.Name

In [213]:
genre_types = items.columns[1:]

In [215]:
items_array = np.array(items)
movie_genre = np.array(items[genre_types])

In [216]:
items_array

array([['Toy Story (1995)', 0, 0, ..., 0, 0, 0],
       ['GoldenEye (1995)', 0, 1, ..., 1, 0, 0],
       ['Four Rooms (1995)', 0, 0, ..., 1, 0, 0],
       ..., 
       ['Sliding Doors (1998)', 0, 0, ..., 0, 0, 0],
       ['You So Crazy (1994)', 0, 0, ..., 0, 0, 0],
       ['Scream of Stone (Schrei aus Stein) (1991)', 0, 0, ..., 0, 0, 0]], dtype=object)

##### Cluster Definition

In [217]:
def FiveStarMovies(centroids, labels, k, NTerms):
    five_star_movies = []
    cluster_movies = []
    for i in range(k):
        cluster_members = ratings[labels==i]
        print 'cluster',i+1,'size: ', cluster_members.shape[0]
        movie_total_rating = np.array(cluster_members.sum(0)).astype(float)
        movie_rating_count = np.array((cluster_members!=0).sum(0)).astype(float)
        movie_average_rating = np.divide(movie_total_rating, movie_rating_count)
        movie_average_rating[np.isnan(movie_average_rating)] = 0
        rated_movie_list = sorted(zip(movie_names, movie_average_rating), key=lambda x: int(x[1]), reverse=True)
        to_append = []
        cluster_append = []
        for j in rated_movie_list:
            if j[1] == 5:
                to_append.append(j[0])   
            if j[1] != 0:
                cluster_append.append(j[0])
        five_star_movies.append(to_append)
        cluster_movies.append(cluster_append)
    print "\n"
    # most rated movies in each cluster
    k = 0
    for m in five_star_movies:
        prototype = list(np.zeros(len(genre_types)))
        for n in items_array:    
            for i in m:
                if i == n[0]:
                    prototype += n[1:]
        print "Representative Movies:"
        for i in range(5):
            print m[i]
        print "\n"
        prototype = map(int, prototype)
        movie_genre_to_use = sorted(zip(genre_types, prototype), key=lambda x:int(x[1]), reverse=True)
        print "Cluster", k+1, "top genre: "
        for i in movie_genre_to_use[:NTerms]:
            print i[0]
        print "\n"
        k += 1
    return five_star_movies, cluster_movies

In [218]:
five_star_movies, cluster_movies = FiveStarMovies(centroids, labels, 5, 5)

cluster 1 size:  163
cluster 2 size:  383
cluster 3 size:  98
cluster 4 size:  65
cluster 5 size:  234


Representative Movies:
Dolores Claiborne (1994)
Three Colors: Red (1994)
So I Married an Axe Murderer (1993)
Dances with Wolves (1990)
D3: The Mighty Ducks (1996)


Cluster 1 top genre: 
Drama
Comedy
Romance
Adventure
Thriller


Representative Movies:
Four Rooms (1995)
Faster Pussycat! Kill! Kill! (1965)
Aristocats, The (1970)
Homeward Bound: The Incredible Journey (1993)
Paradise Lost: The Child Murders at Robin Hood Hills (1996


Cluster 2 top genre: 
Drama
Comedy
Children
Adventure
Romance


Representative Movies:
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Maya Lin: A Strong Clear Vision (1994)
Deceiver (1997)
Meet John Doe (1941)
Telling Lies in America (1997)


Cluster 3 top genre: 
Drama
Crime
Comedy
Documentary
Romance


Representative Movies:
Stalingrad (1993)
Baby-Sitters Club, The (1995)
Timecop (1994)
Great Day in Harlem, A (1994)
Two or Three Things I Know Abou